In [1]:
import json
from pathlib import Path
from itertools import combinations
from typing import Any

import pandas as pd
import numpy as np
import torch
from transformers import (
    AutoModelWithLMHead, AutoTokenizer, pipeline, Pipeline,
    WhisperProcessor, WhisperForConditionalGeneration
)
import pysrt
from IPython.display import clear_output
import IPython.display
import librosa

from asr.asr import (
    initialize_model_for_speech_segmentation, initialize_model_for_speech_classification,
    initialize_model_for_speech_recognition
)
from asr.comparison import MultipleTextsAlignment, filter_correction_suggestions

In [2]:
%cd ../pisets_test_set

/home/oleg/pisets_test_set


/home/oleg/pisets/venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
# import pysrt

# for name in ['galore', 'tuberculosis']:
#     truth = ' '.join([sub.text for sub in pysrt.open(name + '.srt')])
#     with open(name + '.txt', 'w') as f:
#         f.write(truth)

In [ ]:
import librosa
from tqdm.auto import tqdm
from datasets import Dataset, Audio

metainfo = {
    'zaliznyak': {'noise': 'reverberation, background speech', 'domain': 'philology'},
    'harvard': {'noise': 'background speech', 'domain': 'philosophy'},
    'savvateev': {'noise': 'reverberation, background speech', 'domain': 'mathematics'},
    'zhirinovsky': {'noise': 'reverberation, background speech', 'domain': 'politics'},
    'lankov': {'noise': 'reverberation, background speech', 'domain': 'history'},
    'kolodezev': {'noise': 'unknown (TODO)', 'domain': 'machine learning'},
    'tuberculosis': {'noise': 'unknown (TODO)', 'domain': 'medicine'},
}

samples = []

for name in tqdm(metainfo):
    waveform, _ = librosa.load(f'{name}.wav', sr=16_000)
    with open(f'{name}.txt') as f:
        transcription = f.read()

    samples.append({
        'name': name,
        'audio': {'array': waveform, 'sampling_rate': 16_000},
        'transcription': transcription,
        **metainfo[name],
    })

dataset = Dataset.from_list(samples) #.cast_column("audio", Audio(decode=False))

In [ ]:
# https://github.com/huggingface/datasets/issues/6703#issuecomment-1974761165
dataset.to_parquet('long_audio_youtube_dataset/data.parquet')

In [5]:
from datasets import load_dataset
dataset = load_dataset('long_audio_youtube_dataset')
dataset

DatasetDict({
    train: Dataset({
        features: ['name', 'audio', 'transcription', 'noise', 'domain'],
        num_rows: 7
    })
})

In [10]:
from datasets import Audio

dataset.cast_column("audio", Audio(sampling_rate=16_000))['train'][0]['audio']

{'path': None,
 'array': array([0.00042725, 0.00112915, 0.00146484, ..., 0.00222778, 0.00164795,
        0.00262451]),
 'sampling_rate': 16000}